In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/news-articles/Articles.csv
/kaggle/input/thesis2/merged.csv
/kaggle/input/kalpurushfont/Siyamrupali.ttf


In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


CPU times: user 5.21 s, sys: 1.14 s, total: 6.35 s
Wall time: 11.4 s


In [ ]:
import pandas as pd
data = pd.read_csv("/kaggle/input/thesis2/merged.csv")
print(f'Total number of Documents: {len(data)}')

Total number of Documents: 7300


In [ ]:
import pandas as pd

# Assuming your dataset is stored in a DataFrame called 'df'
# Replace 'clean text' with the actual column name in your dataset
duplicates = data[data.duplicated(subset='text', keep=False)]

# 'duplicates' now contains all rows where the 'clean text' column has duplicate values
# You can inspect these entries to understand why they are duplicated

# To print the count of duplicated rows:
print(f"Number of duplicated rows: {len(duplicates)}")

Number of duplicated rows: 0


In [ ]:
data = data[['text','label']]

In [ ]:
data = data.rename(columns={0:'text',1:'label'})

In [ ]:
total_categories = data['label'].nunique()
print("Total number of categories:", total_categories)

Total number of categories: 6


In [ ]:
# Assuming 'data' is your DataFrame with columns renamed
category_names = data['label'].unique()
print("Category names:", category_names)

Category names: ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security '
 'Operating System' 'Cyber Security']


In [ ]:
# Replace 'Cyber Security' and 'Cyber Security ' with 'Cyber Security'
data['label'] = data['label'].replace(['Cyber Security ', 'Cyber Security  '], 'Cyber Security')

# Replace 'Operating System' and 'operating System' with 'Operating System'
data['label'] = data['label'].replace(['Operating System', 'operating System'], 'Operating System')

In [ ]:
# Assuming 'data' is your DataFrame with the 'label' column containing category names
unique_classes = data['label'].unique()
print("Unique classes:", unique_classes)

Unique classes: ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security'
 'Operating System']


In [ ]:
print("First rows of dataset")
data.head()

First rows of dataset


,text,label
0,সফটওয়্যার ইঞ্জিনিয়ারিং দুইটি শব্দ সফটওয়্যার...,Software Engineering
1,সফটওয়্যার কিছু প্রোগ্রামের সমষ্টি,Software Engineering
2,সমস্ত প্রোগ্রাম যখন কোন নির্ধারিত সমস্যা সমাধা...,Software Engineering
3,অপরদিকে ইঞ্জিনিয়ারিং বলতে যে সকল নীতিমালা ও ম...,Software Engineering
4,সফটওয়্যার ইঞ্জিনিয়ারিং বলতে কী বুঝায়,Software Engineering


In [ ]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    #english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    #text=english_pattern.sub(r'', text)
    text = re.sub(r'₀|₁|₂|₃|₄|₅|₆|₇|₈|₉', '', text)
    #text = re.sub(r'⁰|¹|²|³|⁴|⁵|⁶|⁷|⁸|⁹|⁻|⁺|ⁿ|ᵗ', '', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=৳০১২৩৪৫৬৭৮৯012–34567•89।!()-−[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰θ→°π∈∝α≤≥Δβε≠�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out


In [ ]:
data['cleaned'] = data['text'].apply(lambda x: preprocessing(str(x)))

In [ ]:
sample_data = range(4000, 4005)
for i in sample_data:
    print(data['cleaned'][i])

গ্রে হ্যাট হ্যাকাররা  আবার তা নিজের স্বার্থের জন্যও ব্যবহার করতে পারে
ব্ল্যাক হ্যাট হ্যাকার Black Hat Hacker এই ধরনের হ্যাকাররা সবচেয়ে ভয়ংকর প্রকৃতির হ্যাকার 
ব্ল্যাক হ্যাট হ্যাকার  কোনো একটি সিকিউরিটি সিস্টেমের ত্রুটিগুলো বের করলে দ্রুত ঐ ত্রুটিকে নিজের স্বার্থে কাজে লাগায় 
ব্ল্যাক হ্যাট হ্যাকার  এরা সিস্টেম নষ্ট করে বিভিন্ন ভাইরাস ছড়িয়ে দেয় 
ব্ল্যাক হ্যাট হ্যাকার  তারা ভবিষ্যতে নিজে আবার যেন ঢুকতে পারে সে পথ রাখে 


In [ ]:
stop =["অতএব","অথচ","অথবা","অনুযায়ী","অনেক","অনেকে","অনেকেই","অন্তত","অন্য","অবধি","অবশ্য","অর্থাত","আই","আগামী","আগে","আগেই","আছে","আজ","আদ্যভাগে","আপনার","আপনি","আবার","আমরা","আমাকে","আমাদের","আমার","আমি","আর","আরও","ই","ইত্যাদি","ইহা","উচিত","উত্তর","উনি","উপর","উপরে","এ","এঁদের","এঁরা","এই","একই","একটি","একবার","একে","এক্","এখন","এখনও","এখানে","এখানেই","এটা","এটাই","এটি","এত","এতটাই","এতে","এদের","এব","এবং","এবার","এমন","এমনকী","এমনি","এর","এরা","এল","এস","এসে","ঐ","ও","ওঁদের","ওঁর","ওঁরা","ওই","ওকে","ওখানে","ওদের","ওর","ওরা","কখনও","কত","কবে","কমনে","কয়েক","কয়েকটি","করছে","করছেন","করতে","করবে","করবেন","করলে","করলেন","করা","করাই","করায়","করার","করি","করিতে","করিয়া","করিয়ে","করে","করেই","করেছিলেন","করেছে","করেছেন","করেন","কাউকে","কাছ","কাছে","কাজ","কাজে","কারও","কারণ","কি","কিংবা","কিছু","কিছুই","কিন্তু","কী","কে","কেউ","কেউই","কেখা","কেন","কোটি","কোন","কোনও","কোনো","ক্ষেত্রে","কয়েক","খুব","গিয়ে","গিয়েছে","গিয়ে","গুলি","গেছে","গেল","গেলে","গোটা","চলে","চান","চায়","চার","চালু","চেয়ে","চেষ্টা","ছাড়া","ছাড়াও","ছিল","ছিলেন","জন","জনকে","জনের","জন্য","জন্যওজে","জানতে","জানা","জানানো","জানায়","জানিয়ে","জানিয়েছে","জে","জ্নজন","টি","ঠিক","তখন","তত","তথা","তবু","তবে","তা","তাঁকে","তাঁদের","তাঁর","তাঁরা","তাঁাহারা","তাই","তাও","তাকে","তাতে","তাদের","তার","তারপর","তারা","তারৈ","তাহলে","তাহা","তাহাতে","তাহার","তিনঐ","তিনি","তিনিও","তুমি","তুলে","তেমন","তো","তোমার","থাকবে","থাকবেন","থাকা","থাকায়","থাকে","থাকেন","থেকে","থেকেই","থেকেও","দিকে","দিতে","দিন","দিয়ে","দিয়েছে","দিয়েছেন","দিলেন","দু","দুই","দুটি","দুটো","দেওয়া","দেওয়ার","দেওয়া","দেখতে","দেখা","দেখে","দেন","দেয়","দ্বারা","ধরা","ধরে","ধামার","নতুন","নয়","না","নাই","নাকি","নাগাদ","নানা","নিজে","নিজেই","নিজেদের","নিজের","নিতে","নিয়ে","নিয়ে","নেই","নেওয়া","নেওয়ার","নেওয়া","নয়","পক্ষে","পর","পরে","পরেই","পরেও","পর্যন্ত","পাওয়া","পাচ","পারি","পারে","পারেন","পি","পেয়ে","পেয়্র্","প্রতি","প্রথম","প্রভৃতি","প্রযন্ত","প্রাথমিক","প্রায়","প্রায়","ফলে","ফিরে","ফের","বক্তব্য","বদলে","বন","বরং","বলতে","বলল","বললেন","বলা","বলে","বলেছেন","বলেন","বসে","বহু","বা","বাদে","বার","বি","বিনা","বিভিন্ন","বিশেষ","বিষয়টি","বেশ","বেশি","ব্যবহার","ব্যাপারে","ভাবে","ভাবেই","মতো","মতোই","মধ্যভাগে","মধ্যে","মধ্যেই","মধ্যেও","মনে","মাত্র","মাধ্যমে","মোট","মোটেই","যখন","যত","যতটা","যথেষ্ট","যদি","যদিও","যা","যাঁর","যাঁরা","যাওয়া","যাওয়ার","যাওয়া","যাকে","যাচ্ছে","যাতে","যাদের","যান","যাবে","যায়","যার","যারা","যিনি","যে","যেখানে","যেতে","যেন","যেমন","র","রকম","রয়েছে","রাখা","রেখে","লক্ষ","শুধু","শুরু","সঙ্গে","সঙ্গেও","সব","সবার","সমস্ত","সম্প্রতি","সহ","সহিত","সাথে","সাধারণ","সামনে","সি","সুতরাং","সে","সেই","সেখান","সেখানে","সেটা","সেটাই","সেটাও","সেটি","স্পষ্ট","স্বয়ং","হইতে","হইবে","হইয়া","হওয়া","হওয়ায়","হওয়ার","হচ্ছে","হত","হতে","হতেই","হন","হবে","হবেন","হয়","হয়তো","হয়নি","হয়ে","হয়েই","হয়েছিল","হয়েছে","হয়েছেন","হল","হলে","হলেই","হলেও","হলো","হাজার","হিসাবে","হৈলে","হোক","হয়"]
display(stop)

['অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [ ]:
def stopwordRemoval(text):
    x=str(text)
    l=x.split()

    stm=[elem for elem in l if elem not in stop]

    out=' '.join(stm)

    return str(out)

In [ ]:
# Apply the stopwordRemoval function to the 'clean text' column
data['cleaned'] = data['cleaned'].apply(lambda x: stopwordRemoval(str(x)))

# Iterate through the first 1000 rows and print the 'clean text' column values
sample_data = range(0, 5)
for i in sample_data:
    print(data['cleaned'][i])

সফটওয়্যার ইঞ্জিনিয়ারিং দুইটি শব্দ সফটওয়্যার ইঞ্জিনিয়ারিংএর সমন্বয়ে গঠিত
সফটওয়্যার প্রোগ্রামের সমষ্টি
প্রোগ্রাম নির্ধারিত সমস্যা সমাধানের উদ্দেশ্যে তৈরি সফটওয়্যার প্রোডাক্ট
অপরদিকে ইঞ্জিনিয়ারিং সকল নীতিমালা মেথড সমূহ প্রোডাক্টটি Software ফিনিশিং প্রোডাক্ট সেগুলোর সুশৃঙ্খল প্রয়োগ করাকে বুঝায়
সফটওয়্যার ইঞ্জিনিয়ারিং বুঝায়


In [ ]:
# Split the data into training, validation, and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Testing data shape:", test_data.shape)

Training data shape: (5110, 3)
Validation data shape: (1095, 3)
Testing data shape: (1095, 3)


In [ ]:
train_data['enc_label'] = train_data['label'].replace({'Software Engineering':0, 'DBMS':1, 'Data Communication':2 ,'Cyber Security':3,
 'Operating System':4})
val_data['enc_label'] = val_data['label'].replace({'Software Engineering':0, 'DBMS':1, 'Data Communication':2 ,'Cyber Security':3,
 'Operating System':4})

In [ ]:
train_data['cleaned'].tolist()

['ধরনের পরিস্থিতি মোকাবেলা দরকার ইন্টেলেকচুয়াল প্রপার্টি আইনের প্রয়োগ',
 'কম্পিউটার ভাইরাস মূলত প্রোগ্রামারদের ডেভেলপ ক্ষতিকর প্রোগ্রাম',
 'তাছাড়া ভাটা দ্রুত পড়া Read যায়',
 'প্রযুক্তির অনুমতি প্রক্রিয়াকরণ ট্রান্সপন্ডার ত্রুটি কর্মক্ষমতা ডিজিটাল সিগন্যালিংয়ের উন্নতি উপলব্ধি ব্যবহারে আসা',
 'ফাইল মুছে ফেলার নামযুক্ত ফাইলের ডিরেক্টরি অনুসন্ধান',
 'আরেক দল ক্র্যাকার সফটওয়্যার টুলের সিস্টেমে অবৈধভাবে প্রবেশ',
 'পাসিং রেফারেন্স বাদ কথোপকথনের অংশ সফ্টওয়্যার কমই উল্লেখ',
 'মডুলেটেড ক্যারিয়ার ওয়েভের প্রেক্ষিতেও মডুলেশন ইনডেক্সকে সংজ্ঞায়িত যায়',
 'অবজেক্ট অরিয়েন্টেড ডাটা মডেল মূলত অবজেক্ট অরিয়েন্টেড প্রোগ্রামিং ল্যাংগুয়েজের ধারণার প্রতিষ্ঠিত',
 'সিস্টেমটি ব্যবহারযোগ্য দক্ষতার ডেটা পুনরুদ্ধার',
 'কমান্ড পেলে প্রোগ্রাম ডিস্ক মেমোরিতে পূর্বের প্রোগ্রাম ওভাররাইট',
 'সিগন্যালের সেকেন্ডে ডাটা ট্রান্সফারের হারকে Data Rate',
 'কোড লেখার সময় প্রোগ্রামারগণ কোড উদ্দেশ্য কারা লিখছে কোথায় প্রয়োজন',
 'উপাদানস্তরের নকশা সফ্টওয়্যার আর্কিটেকচারের কাঠামোগত উপাদানগুলিকে সফ্টওয়্যার উপাদানগুলির পদ্ধতিগত বিবরণে

In [ ]:
train_data['enc_label'].value_counts()

3    1122
2    1084
0    1004
1     973
4     927
Name: enc_label, dtype: int64

In [ ]:
val_data['enc_label'].value_counts()

2    228
0    225
3    219
1    212
4    211
Name: enc_label, dtype: int64

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
train_data = train_data.rename(columns={'label_Software Engineering':'Software Engineering','label_DBMS':'DBMS',
                                        'label_Data Communication':'Data Communication','label_Cyber Security':'Cyber Security','label_Operating System':'Operating System'})

train_data

,text,label,cleaned,enc_label
4041,এই ধরনের পরিস্থিতি মোকাবেলা করতে দরকার হয় ইন্ট...,Cyber Security,ধরনের পরিস্থিতি মোকাবেলা দরকার ইন্টেলেকচুয়াল প...,3
4258,কম্পিউটার ভাইরাস মূলত প্রোগ্রামারদের ডেভেলপ কর...,Cyber Security,কম্পিউটার ভাইরাস মূলত প্রোগ্রামারদের ডেভেলপ ক্...,3
1748,তাছাড়া এটি হতে ভাটা দ্রুত পড়া (Read) যায়।,DBMS,তাছাড়া ভাটা দ্রুত পড়া Read যায়,1
6581,"প্রযুক্তির অনুমতি হিসাবে, প্রক্রিয়াকরণ ট্রান্...",Data Communication,প্রযুক্তির অনুমতি প্রক্রিয়াকরণ ট্রান্সপন্ডার ...,2
6911,"একটি ফাইল মুছে ফেলার জন্য, আমরা নামযুক্ত ফাইলে...",Operating System,ফাইল মুছে ফেলার নামযুক্ত ফাইলের ডিরেক্টরি অনুস...,4
...,...,...,...,...
5191,স্বয়ংক্রিয়ভাবে এক জব থেকে অন্য জবে যাওয়ার উপর ...,Operating System,স্বয়ংক্রিয়ভাবে এক জব জবে যাওয়ার ভিত্তি অপারেটি...,4
5226,সার্ভার প্রথমে ক্লায়েন্টের নিকট হতে নির্দিষ্ট ...,Operating System,সার্ভার প্রথমে ক্লায়েন্টের নিকট নির্দিষ্ট Part...,4
5390,যদি কোনো System-এ Deadlock prevention বা Deadl...,Operating System,Systemএ Deadlock prevention Deadlock avoidance...,4
860,কম্পিউটার প্রোগ্রামিং ভাষা প্রয়োগের মাধ্যমে সফ...,Software Engineering,কম্পিউটার প্রোগ্রামিং ভাষা প্রয়োগের সফটওয়্যার ...,0


In [ ]:
def char_length(row):
    return len(row.split())

In [ ]:
lables = ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security'
 'Operating System']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].cleaned) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ")
    print("Number of Words:{}".format(len(word_list)))
    print("Number of Unique Words:{}".format(len(np.unique(word_list))))
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))

In [ ]:
# Class Distribution
class_distribution = train_data['label'].value_counts()

# Text Length Distribution
train_data['text_length'] = train_data['cleaned'].apply(lambda x: len(x.split()))
text_length_stats = train_data['text_length'].describe()

print("Class Distribution:")
print(class_distribution)
print("\nText Length Distribution:")
print(text_length_stats)

Class Distribution:
Cyber Security          1122
Data Communication      1084
Software Engineering    1004
DBMS                     973
Operating System         927
Name: label, dtype: int64

Text Length Distribution:
count    5110.000000
mean        9.935225
std         4.734930
min         1.000000
25%         7.000000
50%         9.000000
75%        12.000000
max        59.000000
Name: text_length, dtype: float64


In [ ]:
class_statistics = {}

# Iterate over each unique class
for class_name in train_data['label'].unique():
    # Filter the training data for the current class
    class_data = train_data[train_data['label'] == class_name]

    # Number of Words
    total_words = class_data['cleaned'].apply(lambda x: len(x.split())).sum()

    # Number of Unique Words
    unique_words = len(set(" ".join(class_data['cleaned']).split()))

    # Maximum Post Length
    max_length = class_data['cleaned'].apply(lambda x: len(x.split())).max()

    # Average Words in Texts
    average_length = class_data['cleaned'].apply(lambda x: len(x.split())).mean()

    # Store the statistics for the current class
    class_statistics[class_name] = {
        'Number of Words': total_words,
        'Number of Unique Words': unique_words,
        'Maximum post length': max_length,
        'Average Words in texts': average_length
    }

# Print the statistics for each class
for class_name, stats in class_statistics.items():
    print("Class Name:", class_name)
    for stat_name, value in stats.items():
        print(f"{stat_name}: {value}")
    print()


Class Name: Cyber Security
Number of Words: 11055
Number of Unique Words: 3615
Maximum post length: 59
Average Words in texts: 9.852941176470589

Class Name: DBMS
Number of Words: 9610
Number of Unique Words: 2863
Maximum post length: 38
Average Words in texts: 9.87667009249743

Class Name: Data Communication
Number of Words: 10598
Number of Unique Words: 3532
Maximum post length: 33
Average Words in texts: 9.776752767527675

Class Name: Operating System
Number of Words: 8974
Number of Unique Words: 2990
Maximum post length: 34
Average Words in texts: 9.680690399137001

Class Name: Software Engineering
Number of Words: 10532
Number of Unique Words: 3112
Maximum post length: 57
Average Words in texts: 10.49003984063745



In [ ]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 41.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.41.3-py3-none-any.whl size=25316979 sha256=a65edd726329f8ea11bae75cc99d8182c7d1e5a2072beb9cd2ac268272d890d3
  Stored in directory: /root/.cache/pip/wheels/97/cd/a7/3ef9b6abdab10d64a111bac7f6f1cc4e9ea818c478335fb394
  Created wheel for keras_bert: fil

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import ktrain
from ktrain import text

In [ ]:
train_data['label'].value_counts()

Cyber Security          1122
Data Communication      1084
Software Engineering    1004
DBMS                     973
Operating System         927
Name: label, dtype: int64

In [ ]:
val_data['label'].value_counts()

Data Communication      228
Software Engineering    225
Cyber Security          219
DBMS                    212
Operating System        211
Name: label, dtype: int64

In [ ]:
categories = ['Software Engineering', 'DBMS' ,'Data Communication', 'Cyber Security',
 'Operating System']

In [ ]:
X_train = train_data['cleaned'].tolist()
y_train = train_data['label'].tolist()

X_valid = val_data['cleaned'].tolist()
y_valid = val_data['label'].tolist()

X_test = test_data['cleaned'].tolist()
y_test = test_data['label'].tolist()

In [ ]:
model_name = 'distilbert-base-multilingual-cased'
trans = text.Transformer(model_name,maxlen=100,class_names=categories)

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869M [00:00<?, ?B/s]

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

preprocessing train...
language: bn
train sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 26


Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 25


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=32)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(5)}
weight

{0: 1.0179282868525896,
 1: 1.0503597122302157,
 2: 0.9428044280442804,
 3: 0.910873440285205,
 4: 1.1024811218985977}

In [ ]:
learner.fit_onecycle(2e-5,10,class_weight=weight)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
160/160 [==============================] - 47s 236ms/step - loss: 1.5983 - accuracy: 0.2548 - val_loss: 1.5129 - val_accuracy: 0.4183
Epoch 2/10
160/160 [==============================] - 36s 225ms/step - loss: 1.2657 - accuracy: 0.5225 - val_loss: 0.9574 - val_accuracy: 0.6895
Epoch 3/10
160/160 [==============================] - 36s 225ms/step - loss: 0.7770 - accuracy: 0.7470 - val_loss: 0.6379 - val_accuracy: 0.7863
Epoch 4/10
160/160 [==============================] - 36s 225ms/step - loss: 0.5186 - accuracy: 0.8382 - val_loss: 0.5300 - val_accuracy: 0.8174
Epoch 5/10
160/160 [==============================] - 36s 225ms/step - loss: 0.3763 - accuracy: 0.8838 - val_loss: 0.5453 - val_accuracy: 0.8155
Epoch 6/10
160/160 [==============================] - 36s 225ms/step - loss: 0.2636 - accuracy: 0.9200 - val_loss: 0.4370 - val_accuracy: 0.8575
Epoch 7/10
160/160 [==============================] - 36s 225ms/ste

In [ ]:
learner.validate(class_names=categories)

                      precision    recall  f1-score   support

Software Engineering       0.86      0.85      0.86       219
                DBMS       0.84      0.90      0.87       212
  Data Communication       0.86      0.94      0.90       228
      Cyber Security       0.88      0.79      0.83       211
    Operating System       0.90      0.84      0.87       225

            accuracy                           0.87      1095
           macro avg       0.87      0.86      0.86      1095
        weighted avg       0.87      0.87      0.87      1095



array([[187,   7,  17,   5,   3],
       [  7, 191,   1,  10,   3],
       [  5,   2, 214,   1,   6],
       [  8,  18,  10, 167,   8],
       [ 11,  10,   8,   7, 189]])

In [ ]:
save_path = '/kaggle/working/distilbert'
ktrain.get_predictor(learner.model, preproc=trans).save(save_path)

In [ ]:
# Import necessary packages
from sklearn.metrics import accuracy_score, classification_report
import ktrain

# Specify the file path for saving and loading the model in the Kaggle working directory
save_path = '/kaggle/working/distilbert'

# Load the predictor from the specified path
predictor = ktrain.load_predictor(save_path)

# Assuming you have your test data available as 'X_test' and 'y_test' in your Kaggle environment
# Make predictions on the test data using the loaded predictor
y_pred = predictor.predict(X_test)

# Calculate accuracy by comparing the predicted labels with true labels
accuracy = accuracy_score(y_test, y_pred)
print(f"Test data accuracy: {accuracy:.2%}")

# Generate classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print("Classification Report:")
print(report)


Test data accuracy: 84.29%
Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.86      0.83      0.84       243
                DBMS       0.86      0.80      0.83       216
  Data Communication       0.80      0.87      0.84       222
    Operating System       0.86      0.87      0.87       215
Software Engineering       0.83      0.84      0.84       199

            accuracy                           0.84      1095
           macro avg       0.84      0.84      0.84      1095
        weighted avg       0.84      0.84      0.84      1095



In [ ]:
model_name = 'sagorsarker/bangla-bert-base'
trans = text.Transformer(model_name,maxlen=100,class_names=categories)

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/930M [00:00<?, ?B/s]

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

preprocessing train...
language: bn
train sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 26


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 25


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=32)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(5)}
weight

{0: 1.0179282868525896,
 1: 1.0503597122302157,
 2: 0.9428044280442804,
 3: 0.910873440285205,
 4: 1.1024811218985977}

In [ ]:
learner.fit_onecycle(2e-5,10,class_weight=weight)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
160/160 [==============================] - 91s 444ms/step - loss: 1.4379 - accuracy: 0.4213 - val_loss: 1.0958 - val_accuracy: 0.6457
Epoch 2/10
160/160 [==============================] - 68s 422ms/step - loss: 0.8440 - accuracy: 0.7178 - val_loss: 0.6664 - val_accuracy: 0.7744
Epoch 3/10
160/160 [==============================] - 67s 422ms/step - loss: 0.5330 - accuracy: 0.8192 - val_loss: 0.5647 - val_accuracy: 0.8082
Epoch 4/10
160/160 [==============================] - 68s 422ms/step - loss: 0.3601 - accuracy: 0.8798 - val_loss: 0.5234 - val_accuracy: 0.8183
Epoch 5/10
160/160 [==============================] - 67s 422ms/step - loss: 0.2524 - accuracy: 0.9202 - val_loss: 0.5364 - val_accuracy: 0.8301
Epoch 6/10
160/160 [==============================] - 68s 422ms/step - loss: 0.1497 - accuracy: 0.9534 - val_loss: 0.5676 - val_accuracy: 0.8301
Epoch 7/10
160/160 [==============================] - 68s 422ms/ste

In [ ]:
learner.validate(class_names=categories)

                      precision    recall  f1-score   support

Software Engineering       0.89      0.88      0.89       219
                DBMS       0.79      0.86      0.82       212
  Data Communication       0.86      0.88      0.87       228
      Cyber Security       0.82      0.75      0.78       211
    Operating System       0.89      0.86      0.87       225

            accuracy                           0.85      1095
           macro avg       0.85      0.85      0.85      1095
        weighted avg       0.85      0.85      0.85      1095



array([[193,  13,   7,   4,   2],
       [  5, 183,   6,  13,   5],
       [  7,   2, 200,  13,   6],
       [  6,  18,  16, 159,  12],
       [  6,  16,   4,   6, 193]])

In [ ]:
save_path = '/kaggle/working/sagorsarker'
ktrain.get_predictor(learner.model, preproc=trans).save(save_path)

The OrderedVocab you are attempting to save contains a hole for index 1015, your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains a hole for index 1016, your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains a hole for index 1017, your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains a hole for index 1018, your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains a hole for index 1053, your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains a hole for index 1054, your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains a hole for index 1055, your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains a hole for index 1056, your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains a hole for index 1057, your vocabulary coul

In [ ]:
# Import necessary packages
from sklearn.metrics import accuracy_score, classification_report
import ktrain

# Specify the file path for saving and loading the model in the Kaggle working directory
save_path = '/kaggle/working/sagorsarker'

# Load the predictor from the specified path
predictor = ktrain.load_predictor(save_path)

# Assuming you have your test data available as 'X_test' and 'y_test' in your Kaggle environment
# Make predictions on the test data using the loaded predictor
y_pred = predictor.predict(X_test)

# Calculate accuracy by comparing the predicted labels with true labels
accuracy = accuracy_score(y_test, y_pred)
print(f"Test data accuracy: {accuracy:.2%}")

# Generate classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print("Classification Report:")
print(report)


Test data accuracy: 83.56%
Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.90      0.83      0.86       243
                DBMS       0.80      0.80      0.80       216
  Data Communication       0.83      0.87      0.85       222
    Operating System       0.78      0.83      0.80       215
Software Engineering       0.87      0.85      0.86       199

            accuracy                           0.84      1095
           macro avg       0.84      0.84      0.84      1095
        weighted avg       0.84      0.84      0.84      1095



In [ ]:
model_name = 'ai4bharat/indic-bert'
trans = text.Transformer(model_name,maxlen=100,class_names=categories)

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

preprocessing train...
language: bn
train sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 26


Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 10
	95percentile : 19
	99percentile : 25


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=32)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(5)}
weight

{0: 1.0179282868525896,
 1: 1.0503597122302157,
 2: 0.9428044280442804,
 3: 0.910873440285205,
 4: 1.1024811218985977}

In [ ]:
learner.fit_onecycle(2e-5,10,class_weight=weight)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
160/160 [==============================] - 81s 406ms/step - loss: 1.6151 - accuracy: 0.2284 - val_loss: 1.5978 - val_accuracy: 0.2804
Epoch 2/10
160/160 [==============================] - 62s 389ms/step - loss: 1.4783 - accuracy: 0.3593 - val_loss: 1.3498 - val_accuracy: 0.4639
Epoch 3/10
160/160 [==============================] - 62s 389ms/step - loss: 1.1760 - accuracy: 0.5207 - val_loss: 1.1394 - val_accuracy: 0.5927
Epoch 4/10
160/160 [==============================] - 62s 388ms/step - loss: 0.9017 - accuracy: 0.6812 - val_loss: 0.8779 - val_accuracy: 0.6922
Epoch 5/10
160/160 [==============================] - 62s 389ms/step - loss: 0.6676 - accuracy: 0.7734 - val_loss: 0.7524 - val_accuracy: 0.7406
Epoch 6/10
160/160 [==============================] - 62s 388ms/step - loss: 0.4687 - accuracy: 0.8452 - val_loss: 0.6637 - val_accuracy: 0.7799
Epoch 7/10
160/160 [==============================] - 62s 389ms/ste

In [ ]:
learner.validate(class_names=categories)

                      precision    recall  f1-score   support

Software Engineering       0.71      0.80      0.75       219
                DBMS       0.79      0.80      0.79       212
  Data Communication       0.78      0.79      0.79       228
      Cyber Security       0.76      0.72      0.74       211
    Operating System       0.86      0.77      0.81       225

            accuracy                           0.78      1095
           macro avg       0.78      0.78      0.78      1095
        weighted avg       0.78      0.78      0.78      1095



array([[175,   7,  27,   6,   4],
       [ 11, 169,   5,  22,   5],
       [ 20,  16, 181,   6,   5],
       [ 13,  18,  15, 152,  13],
       [ 29,   5,   3,  15, 173]])

In [ ]:
# Save the predictor directly in the working directory
save_path = '/kaggle/working/indicbert'
ktrain.get_predictor(learner.model, preproc=trans).save(save_path)

In [ ]:
# Import necessary packages
from sklearn.metrics import accuracy_score, classification_report
import ktrain

# Specify the file path for saving and loading the model in the Kaggle working directory
save_path = '/kaggle/working/indicbert'

# Load the predictor from the specified path
predictor = ktrain.load_predictor(save_path)

# Assuming you have your test data available as 'X_test' and 'y_test' in your Kaggle environment
# Make predictions on the test data using the loaded predictor
y_pred = predictor.predict(X_test)

# Calculate accuracy by comparing the predicted labels with true labels
accuracy = accuracy_score(y_test, y_pred)
print(f"Test data accuracy: {accuracy:.2%}")

# Generate classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print("Classification Report:")
print(report)

Test data accuracy: 79.45%
Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.78      0.81      0.80       243
                DBMS       0.75      0.75      0.75       216
  Data Communication       0.83      0.81      0.82       222
    Operating System       0.76      0.80      0.78       215
Software Engineering       0.85      0.81      0.83       199

            accuracy                           0.79      1095
           macro avg       0.80      0.79      0.80      1095
        weighted avg       0.80      0.79      0.79      1095

